<a href="https://colab.research.google.com/github/notimeforcaution/airelius/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### a(i)relius

In [4]:
BASE_PATH = "/content/drive/MyDrive/colab_notebooks/airelius/"

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [33]:
# Getting data and tokenizing.

with open(BASE_PATH + "data/meditations.txt", "r") as f:
  text = f.read()

inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)


In [46]:
# Creating Dataset

from torch.utils.data import Dataset

class TextDataset(Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    item = {key: val[idx] for key, val in self.encodings.items()}
    item["labels"] = item["input_ids"][:-1].clone()
    item["input_ids"] = item["input_ids"][1:].clone()  # Next sequence is the output.
    return item

  def __len__(self):
    return len(self.encodings["input_ids"])

train_dataset = TextDataset(inputs)

In [48]:
# Set-up training args.

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=20,
    save_steps=10_000,
    save_total_limit=2,
    logging_strategy="epoch",
)

In [50]:
# Set-up trainer.

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

In [51]:
trainer.train()  # TODO(ntfc): Tune the model better, reduce loss.

Step,Training Loss
1,7.255700
2,6.322500
3,5.444600
4,4.278300
5,3.205700
6,2.278200
7,1.429400
8,0.845500
9,0.685800
10,0.479800


TrainOutput(global_step=20, training_loss=1.7573212742805482, metrics={'train_runtime': 557.2459, 'train_samples_per_second': 0.036, 'train_steps_per_second': 0.036, 'total_flos': 10441474560000.0, 'train_loss': 1.7573212742805482, 'epoch': 20.0})

### Generating

In [65]:
prompt = "Based on your training, generate a novel philosophy quote."
input_ids = tokenizer(prompt, return_tensors="pt", padding=True).input_ids

output = model.generate(
  input_ids,
  # attention_mask=input.attention_mask,
  max_length=100,
  num_beams=5,
  no_repeat_ngram_size=2,
  early_stopping=True
)

# Instead of slicing, filter out invalid token IDs
output = output[:, 0]  # Get the first sequence in the batch
valid_token_ids = [token_id for token_id in output.tolist() if token_id < tokenizer.vocab_size]

print(tokenizer.decode(valid, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### Notes

##### 08-02-2024
Replaced the backend with "t5-base". Stopped at a loss of 0.24 with 20 epochs.